In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200

In [8]:
train = pd.read_csv('train_en.tsv', delimiter='\t')
valid = pd.read_csv('dev_en.tsv', delimiter='\t')
train.sample(5)

,sentence_id,sentence,label,solved_conflict
41,5d8521c0-ea65-425e-94ce-fb73ba1ec702,"It is not entirely clear how BA.4.6 has emerged, but it’s possible it could be a recombinant variant.",OBJ,False
377,5d98fc12-57e1-449f-9ebe-d8d67594a817,A gap has opened up between the gdp path envisaged for China at the start of this year and the grimmer one that now seems probable.,SUBJ,True
169,48d6a8f5-3c17-4e70-b91c-df2b8cb46e1b,"In the minutes of the Chicago City Council, May 12th last is the perfect example of how commonly we regard public credit.",SUBJ,False
485,e746564d-918d-41e3-9e50-b46f0304ce7a,And our society’s approach to addressing important social questions such as climate change and racial inequity are more heavily influenced by the dictates of corporate chiefs in Davos than they ar...,SUBJ,False
756,843e214f-9632-4357-a9dd-38512cdd30f5,"“Cities have assumed new obligations,” writes Lent Upson, director of the Detroit Bureau of Governmental Research.",OBJ,False


In [3]:
print(f"{298/(532+298)*100:.1f}% er subjektive")
train['label'].value_counts()

35.9% er subjektive


OBJ     532
SUBJ    298
Name: label, dtype: int64

In [7]:
print(f"{106/(113+106)*100:.1f}% er subjektive")
valid['label'].value_counts()

48.4% er subjektive


SUBJ    113
OBJ     106
Name: label, dtype: int64

Hvad er "solved conflict"?

Åbenbart True, hvis dem, der har annoteret det har været uenige om svaret - Heraf kan vi allerede sige, at objektivitet er svært at bedømme.

## Eksempel med RF og TF-IDF

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

In [100]:
tfidf = TfidfVectorizer(stop_words='english')
X_train = tfidf.fit_transform(train['sentence']).toarray()
X_test = tfidf.transform(valid['sentence']).toarray()

In [101]:
rfc = RandomForestClassifier()
rfc.fit(X_train, train['label'])

RandomForestClassifier()

In [102]:
preds = rfc.predict(X_test)
rfc.score(X_test, valid['label']) #accuracy

0.5205479452054794

In [103]:
f1_score(valid['label'], preds, average='macro')

0.46939105152640936

In [104]:
dum = DummyClassifier()
dum.fit(X_train, train['label'])

DummyClassifier()

In [105]:
preds = dum.predict(X_test)
dum.score(X_test, valid['label']) #accuracy

0.4840182648401826

In [106]:
f1_score(valid['label'], preds, average='macro')

0.3261538461538462

In [107]:
tfidf.get_feature_names_out().shape

(3848,)